In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data 
from torch.autograd import Variable

In [5]:
#load the data
movies = pd.read_csv("./data/ml-1m/movies.dat", sep = "::", header=None, engine='python', encoding='latin-1')
users = pd.read_csv("./data/ml-1m/users.dat", sep = "::", header=None, engine='python', encoding='latin-1')
ratings = pd.read_csv("./data/ml-1m/ratings.dat", sep = "::", header=None, engine='python', encoding='latin-1')

In [6]:
movies[:5]

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
users[:5]

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [8]:
ratings[:5]

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [11]:
#Prepare training and test set
train_set = pd.read_csv("./data/ml-100k/u1.base", delimiter='\t')
train_set = np.array(train_set, dtype='int')

In [13]:
train_set[:5]

array([[        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       [        1,         5,         3, 889751712],
       [        1,         7,         4, 875071561]])

In [14]:
test_set = pd.read_csv("./data/ml-100k/u1.test", delimiter='\t')
test_set = np.array(test_set, dtype='int')
test_set[:5]

array([[        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       [        1,        14,         5, 874965706],
       [        1,        17,         3, 875073198],
       [        1,        20,         4, 887431883]])

In [15]:
#total user and movies
nb_users = int(max(max(train_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(train_set[:,1]), max(test_set[:,1])))

In [16]:
#Convert data such that users are in rows and movies in columns
def convert(data):
    new_data = []
    for id_users in range(1, nb_users+1):
        id_movies = data[:, 1][data[:, 0] == id_users]
        id_ratings = data[:, 2][data[:, 0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [17]:
train_set = convert(train_set)
test_set = convert(test_set)

In [18]:
#convert data into torch tensors
train_set = torch.FloatTensor(train_set)
test_set = torch.FloatTensor(test_set)

In [19]:
#Create a newral network architecture - Autoencoder
class SAE(nn.Module):
    def __init__(self):
        super(SAE,self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    
    def forward(self, x):
        x = self.activation(self.fc1(x)) # First encoding from input
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

In [20]:
sae = SAE()

In [21]:
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

In [24]:
#Training SAE
nb_epoch = 200
for epoch in range(1, nb_epoch+1):
    train_loss = 0.
    counter = 0
    for id_user in range(nb_users):
        input = Variable(train_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/ float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data[0]*mean_corrector)
            counter += 1
            optimizer.step()
    print('epoch: '+ str(epoch) + ' Loss: ' + str(train_loss/counter))

epoch: 1Loss: 1.76632954092
epoch: 2Loss: 1.09639336454
epoch: 3Loss: 1.05349361746
epoch: 4Loss: 1.03828138331
epoch: 5Loss: 1.03097391166
epoch: 6Loss: 1.02648157503
epoch: 7Loss: 1.02391810081
epoch: 8Loss: 1.02181320818
epoch: 9Loss: 1.02071473789
epoch: 10Loss: 1.0199012669
epoch: 11Loss: 1.01893917302
epoch: 12Loss: 1.01829102679
epoch: 13Loss: 1.01787672019
epoch: 14Loss: 1.01752504983
epoch: 15Loss: 1.01719343309
epoch: 16Loss: 1.01694731622
epoch: 17Loss: 1.01679829112
epoch: 18Loss: 1.01641970477
epoch: 19Loss: 1.0163903684
epoch: 20Loss: 1.01628627154
epoch: 21Loss: 1.01602892246
epoch: 22Loss: 1.01589279739
epoch: 23Loss: 1.0160285077
epoch: 24Loss: 1.01592044782
epoch: 25Loss: 1.01580162347
epoch: 26Loss: 1.01551361134
epoch: 27Loss: 1.01517431046
epoch: 28Loss: 1.01509100619
epoch: 29Loss: 1.01302391856
epoch: 30Loss: 1.0119296137
epoch: 31Loss: 1.00996476006
epoch: 32Loss: 1.00935208517
epoch: 33Loss: 1.00579951506
epoch: 34Loss: 1.00574159027
epoch: 35Loss: 1.0016638150

In [27]:
#testing SAE
test_loss = 0.
counter = 0
for id_user in range(nb_users):
    input = Variable(train_set[id_user]).unsqueeze(0) #visible observations remain same
    target = Variable(test_set[id_user])#actual ratings in test set
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/ float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data[0]*mean_corrector)
        counter += 1
print('Test Loss: ' + str(test_loss/counter))

Test Loss: 0.961124987614
